Name of Members:

- __APA, GIUSIPPI MARIA II DEL ROSARIO__
- __OBEJERO, REY FERBS MAGALLON__
- __RAMIREZ, BENMAR SIM GREFALDA__
- __RAMOS, RONN PATRICK BICERA__

## I. Introduction

The dataset we chose for this machine project is the "Sports" dataset. 

Our project will have us go through the process of selecting a dataset, describing it, performing EDA, data preprocessing and cleaning, model training, HP tuning, model selection, and extracting insights from the data.

We aim to leverage the data from the soccer matches to preduct the number of goals that will be scored during the remaining playtime. The target of this project is regression as we aim to predict a continous outcome, which is the **number of goals to be scored** during the remaining time.

We will implement the use of machine learning and other techniques taught in class to build our predictive model.

## II. Description

The dataset is related to soccer matches and will be used to predict how many goals will be scored during the remaining match time. The dataset contains 10,000 such snapshots, each taken at a different, random point in time during a match. 

In the context of this dataset, a “snapshot” refers to a set of data points captured at a specific moment during a soccer match.

As for the collection process, it can be inferred that the data might have been collected in real-time during this soccer match, capturing various events and statistics at different timestamps. This data would allow for more accurate predictions of future events based on the current state of the match, and whether it is best to bet on over or under a certain number of goals.

There are only 7000 instances(rows) and 21 features(columns) in the dataset. Each of the features are statistics of both events for the home and away team.

### List of Variables

| **Variable Name** | **Description**|
|--------------------------------------|----------------|
|**uuid** | Snapshot’s unique ID|
|**current_minute** | Match’s current minute, including 15 min half-time|
|**home_score and away_score** | Home and away team’s goals.|
|**home_yellow_cards and away_yellow_cards**| Home and away team’s yellow cards.|
|**home_red_cards and away_red_cards**| Home and away team’s red cards.|
|**home_attacks and away_attacks**| Home and away team’s attempted attacks.|
|**home_dangerous_attacks and away_dangerous_attacks**| Home and away team’s dangerous attacks.|
|**home_corners and away_corners**| Home and away team’s awarded corners.|
|**home_off_target and away_off_target**| Home and away team’s off-target shots.|
|**home_on_target and away_on_target**| Home and away team’s on-target shots.|
|**home_possession and away_possession** |Home and away team’s ball possession %.|

## III. Modules

For this machine project, we will utilize the following Python libraries:

- `one`:
- `two`:
- `three`:
- `four`:

In [ ]:
import csv
import h5py
import math
import os
import pickle
import random
from linear_regression import AnalyticalMethod, compute_RMSE
from linear_regression import PolyFitMethod
import matplotlib.pyplot as plt
import numpy as np
from neural_network import NeuralNetwork
import pandas as pd
from scipy import misc
from sklearn import preprocessing
from sklearn.base import BaseEstimator
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import log_loss, accuracy_score, classification_report
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_predict, cross_val_score, ParameterGrid
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
import torch.optim as optim
import torch.nn as nn


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## IV. Data Cleaning

We will now be performing various methods of preprocessing and data cleaning to enhance the dataset's usability and suitability to our exploratory analysis and model training.

This is also to ensure that the dataset will be ready for use, which means we will be checking for missing values, duplicates, outliers and other errors. Making it so that the data is in its correct representation.

In [ ]:
# Importing and reading the Sports.csv
sports = pd.read_csv("sports.csv")
sports.head()

**Checking for Missing Values**



In [ ]:
# Check each column for missing values
missing_values = sports.isnull().sum()

columns_with_missing_values = missing_values[missing_values > 0]
print(columns_with_missing_values)

**Checking for Duplicates**



In [ ]:
# Find the duplicates
duplicate_uuids = sports.duplicated(keep=False)

# Create a dataframe for duplicates
duplicates = sports[duplicate_uuids]

# Display the rows with duplicates
print("Duplicate Entries:\n", duplicates)

# Removing the duplicates if detected in dataframe
if not duplicates.empty:
    sports = sports.drop_duplicates(keep='first')
    print("Duplicates removed. Dataset has {sports.shape[0]} rows.")
else:
    print("No duplicates found.")

**Other Cleaning Processes**

In [ ]:
# Replacing any erroneous infinite values
sports = sports.replace([np.inf, -np.inf], np.nan)

# Dropping columns with more than 50% missing values
sports = sports.dropna(thresh=len(sports)*0.5, axis=1)

# Check for incorrect possession percentages
incorrect_possession_indices = sports[~(sports['home_possession'] + sports['away_possession'] == 100)].index

if len(incorrect_possession_indices) > 0:
    print("Found inconsistencies in possession data:")
    print(sports.loc[incorrect_possession_indices, ['home_possession', 'away_possession']])
else:
    print("Possession data is consistent.")

As you can see there are percentages that don't equate to 100, but the difference is neglible because it is only from 99 to 100 which won't cause much noise in the analysis. It isn't exactly necessary to remove it at this point of the dataset but is also another consideration we took to prepare the if ever the dataset grew larger.

**Data Type Checking**

Having appropriate data types for the values in the dataset is crucial to data analysis and model training. Luckily for us, the dataset we currently have has the appropriate values as all of the variables and their values are appropriate to their data type.

The importance of having correct data types is that we have the following:

Data Integrity,
Accuracy,
Efficiency,
Compatibility, 
and Reliability

In [ ]:
# Printing all of the present data types
print("All data types present:\n", sports.dtypes)

### Outlier Detection and Treatment

Outliers can significantly impact statistical measures and model performance. In the context of our dataset, outliers might represent unusual game conditions, errors in data collection, or truly exceptional events. 

Surprisingly enough, there are no major outliers in our dataset, because in the dataset, varying values like attacks, cards, and accuracy are all subject to the team's playstyle and ability. 

**Our Approach**

But we performed winsorization on the dataset to make it robust against potential outliers in the future if ever there are any in the future, possibly when the dataset grows larger. By capping the data at specified percentiles we reduce the possible influed of these outliers. 

We also provided the before and after outlier treatment visualizations to identify the outliers (if there are any).

In [ ]:
print(sports.head())

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

# Load data from CSV
sports = pd.read_csv('sports.csv')

# Columns to check for outliers
outlier_columns = ['current_minute', 'home_yellow_cards', 'away_yellow_cards', 'home_on_target',
                   'away_on_target', 'home_attacks', 'away_attacks', 'home_dangerous_attacks',
                   'away_dangerous_attacks', 'home_corners', 'away_corners', 'home_off_target',
                   'away_off_target', 'home_possession', 'away_possession']

def winsorize(data, lower_percentile=5, upper_percentile=95):
    """Winsorizes data by capping values at specified percentiles."""
    lower_bound = np.percentile(data, lower_percentile)
    upper_bound = np.percentile(data, upper_percentile)
    return np.clip(data, lower_bound, upper_bound)

def truncate(data, upper_bound):
    """Truncates data by removing values exceeding the upper bound."""
    return data[data <= upper_bound]

def detect_possession_outliers(data, method='zscore', z_score_threshold=3, lower_bound=20, upper_bound=80):
    """
    Identifies outliers in home and away possession data using different methods.

    Args:
        data: Pandas DataFrame containing home_possession and away_possession columns.
        method: The outlier detection method to use ('zscore', 'percentile', 'domain').
        z_score_threshold: Threshold for Z-scores (used with 'zscore' method).
        lower_bound: Lower threshold for possession values (used with 'domain' method).
        upper_bound: Upper threshold for possession values (used with 'domain' method).

    Returns:
        A new DataFrame with additional columns indicating potential outliers.
    """
    home_possession = data['home_possession']
    away_possession = data['away_possession']

    if method == 'zscore':
        # Z-score method (not used for normalization)
        pass
    elif method == 'percentile':
        # Percentile-based method (not used for normalization)
        pass
    elif method == 'domain':
        # Domain knowledge-based method
        data['home_possession_outlier'] = np.where((home_possession < lower_bound) | (home_possession > upper_bound), True, False)
        data['away_possession_outlier'] = np.where((away_possession < lower_bound) | (away_possession > upper_bound), True, False)
    else:
        raise ValueError("Invalid outlier detection method. Choose 'zscore', 'percentile', or 'domain'.")

    return data

# Handle outliers for other columns
for col in outlier_columns:
    if col == 'current_minute':
        sports[col] = truncate(sports[col], 105)
    elif col.endswith('_cards'):
        sports[col] = winsorize(sports[col], upper_percentile=80)
    else:
        sports[col] = winsorize(sports[col])

# Choose a method for possession outliers and apply it
sports = detect_possession_outliers(sports.copy(), method='domain', lower_bound=20, upper_bound=80)  # Example: using domain-based method

# Calculate goals scored after current minute
def calculate_remaining_goals(row):
    total_goals = row['home_score'] + row['away_score']
    goals_so_far = total_goals - row['home_score'] - row['away_score']
    return total_goals - goals_so_far

sports['goals_remaining'] = sports.apply(calculate_remaining_goals, axis=1)

# Calculate additional features
epsilon = 1e-10  # Add a small number to avoid division by zero
sports['home_attack_efficiency'] = sports['home_score'] / (sports['home_attacks'] + epsilon)
sports['away_attack_efficiency'] = sports['away_score'] / (sports['away_attacks'] + epsilon)

sports['home_shot_conversion_rate'] = sports['home_score'] / (sports['home_on_target'] + sports['home_off_target'] + epsilon)
sports['away_shot_conversion_rate'] = sports['away_score'] / (sports['away_on_target'] + sports['away_off_target'] + epsilon)

sports['possession_differential'] = sports['home_possession'] - sports['away_possession']

# Select columns for normalization (excluding uuid)
columns_to_normalize = [col for col in sports.columns if col != 'uuid']

# Create a MinMaxScaler object (normalizes to -1 to 1 range)
scaler = MinMaxScaler(feature_range=(0, 1))

# Apply normalization to selected columns
sports[columns_to_normalize] = scaler.fit_transform(sports[columns_to_normalize])

# Visualize distributions after outlier treatment and normalization
fig, axes = plt.subplots(nrows=len(outlier_columns) + 2, ncols=2, figsize=(15, 40))

for i, col in enumerate(outlier_columns):
    # Box plots
    sns.boxplot(data=sports, x=col, ax=axes[i, 0])
    axes[i, 0].set_title(f'{col} - Before Treatment')

    sns.boxplot(data=sports, x=col, ax=axes[i, 1])
    axes[i, 1].set_title(f'{col} - After Treatment')

# Visualize possession data with outliers
sns.boxplot(data=sports, x='home_possession', ax=axes[len(outlier_columns), 0], hue='home_possession_outlier')
sns.boxplot(data=sports, x='away_possession', ax=axes[len(outlier_columns), 1], hue='away_possession_outlier')

# Histograms for possession data with outliers
sns.histplot(data=sports, x='home_possession', kde=True, ax=axes[len(outlier_columns) + 1, 0], hue='home_possession_outlier')
sns.histplot(data=sports, x='away_possession', kde=True, ax=axes[len(outlier_columns) + 1, 1], hue='away_possession_outlier')

plt.tight_layout()
plt.show()


In [ ]:
print(sports.head())

We did boxplots and histograms for all relevant variables and utilized truncation and winsorization. 

In our current case, there are no obvious outliers, because the initial data might not have contained extreme values that would significantly impact the results. There is also the idea of playstyle variations where the teams at different current_minutes would have varying possessions, attacks, or accuracy.

Despite this, still applying outlier detection methods still offers a safety net from future outliners. We also applied normalization to bring the features to a common scale between 0 and 1, this will help us ensure that there is feature scaling and improved algorithm performance from algorithms like such as neural networks and gradient descent.

### Data Transformation

**Feature Engineering**
`goals_remaining`: This is our target variable, representing the number of goals scored in the remaining time of the match.

Ratio of Attack to Goals

`home_attack_efficiency`: Measures the efficiency of the home team in converting attacks into goals.
`away_attack_efficiency`: Measures the efficiency of the away team in converting attacks into goals.

Ratio of Shots to Goals

`home_shot_conversion_rate`: Measures the efficiency of the home team in converting shots into goals.
`away_shot_conversion_rate`: Measures the efficiency of the away team in converting shots into goals.

In [ ]:
# Calculate goals scored after current minute
def calculate_remaining_goals(row):
    total_goals = row['home_score'] + row['away_score']
    goals_so_far = total_goals - row['home_score'] - row['away_score']
    return total_goals - goals_so_far

sports['goals_remaining'] = sports.apply(calculate_remaining_goals, axis=1)

# Calculate additional features
sports['home_attack_efficiency'] = sports['home_score'] / sports['home_attacks']
sports['away_attack_efficiency'] = sports['away_score'] / sports['away_attacks']
sports['home_shot_conversion_rate'] = sports['home_score'] / (sports['home_on_target'] + sports['home_off_target'])
sports['away_shot_conversion_rate'] = sports['away_score'] / (sports['away_on_target'] + sports['away_off_target'])

print(sports.head())

**Dropping Features**

The uuid column is a unique identifier that does not provide any predictive information, therefore it's correct to drop it.

In [ ]:
# Dropping irrelevant column
sports = sports.drop('uuid', axis=1)

**Correlation Matrix**

**Removal of Features**

## Exploratory Data Analysis

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import csv
import math

# set default size of plots
plt.rcParams['figure.figsize'] = (6.0, 6.0)
plt.rcParams['image.interpolation'] = 'nearest'

%load_ext autoreload
%autoreload 2

## Model Training

## Error Analysis

## Improving Model

## Model Selection

## Insights and Conclusions

## Reference